#まとめ

* train (最初の 3 週間)
* valid A（第4週の前半）
* valid B（第4週の後半）
* all train (train + validA + validB)
* test (第5週の前半)
* LB (第5週の後半)


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd

In [3]:
TRAIN_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-train-and-test-data-for-local-validation/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/'

df_train = pd.read_parquet(TRAIN_DIR+'train.parquet')
df_val = pd.read_parquet(TRAIN_DIR+'test.parquet')

print(len(df_train))
print(len(df_val))

163955180
7683577


In [5]:
df_val_part = df_val.iloc[:10]
df_val_part.columns

Index(['session', 'aid', 'ts', 'type'], dtype='object')

# step1 sessionのごとにitem候補生成 : valid A 

covis matrix読み込み

In [7]:
%%time
VER = 6
DISK_PIECES = 4

def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
top_20_clicks = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt') ) )

CPU times: user 38.5 s, sys: 3.38 s, total: 41.8 s
Wall time: 41.9 s


In [8]:
import collections
from collections import Counter

def generate_candidate(df):
  aids = df.aid.tolist()
  candidates = []
  for k in aids:
        if k in top_20_clicks.keys():
            candidates.extend(top_20_clicks[k])
        else:
            pass
  output_candidateb = [k for k,v in collections.Counter(candidates).most_common(20)]
  
  return output_candidateb

In [9]:
df_val

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [10]:
%%time

candidates = df_val.groupby('session').apply(generate_candidate).explode()
candidates

CPU times: user 2min 45s, sys: 1.66 s, total: 2min 46s
Wall time: 2min 46s


session
11098528     588923
11098528    1732105
11098528     571762
11098528     884502
11098528    1157882
             ...   
12899778      13568
12899778    1097813
12899778    1102546
12899778    1068655
12899778    1495496
Length: 35715483, dtype: object

In [11]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,588923
1,11098528,1732105
2,11098528,571762
3,11098528,884502
4,11098528,1157882
...,...,...
35715478,12899778,13568
35715479,12899778,1097813
35715480,12899778,1102546
35715481,12899778,1068655


In [12]:
candidates[:40]

,session,item
0,11098528,588923
1,11098528,1732105
2,11098528,571762
3,11098528,884502
4,11098528,1157882
5,11098528,876129
6,11098528,1182614
7,11098528,1790438
8,11098528,307904
9,11098528,231487


# step2 item特徴 : train + valid A

In [13]:
df_train

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
163955175,11098523,175715,1661119197,0
163955176,11098524,1088524,1661119198,0
163955177,11098525,182927,1661119199,0
163955178,11098526,510055,1661119199,0


In [14]:
%%time
item_features = pd.merge(df_train, df_val, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
item_features.to_parquet('item_features.pqt')
item_features

CPU times: user 2min 58s, sys: 15 s, total: 3min 13s
Wall time: 3min 12s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


# step3 user特徴 : valid A

In [15]:

user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000
...,...,...,...
12899774,1,1,0.000000
12899775,1,1,0.000000
12899776,1,1,0.000000


# step4 : valid A の user × item 特徴量生成？

# step5 : step1に step2,3を追加

In [16]:
candidates = candidates.rename(columns={'session':'user'})

item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio
0,11098528,588923,23956.0,14293.0,0.124937,1,1,0.0
1,11098528,1732105,10121.0,5565.0,0.280802,1,1,0.0
2,11098528,571762,18185.0,11918.0,0.125763,1,1,0.0
3,11098528,884502,30419.0,17385.0,0.135738,1,1,0.0
4,11098528,1157882,27559.0,15556.0,0.137269,1,1,0.0
...,...,...,...,...,...,...,...,...
35715478,12899778,13568,855.0,527.0,0.173099,1,1,0.0
35715479,12899778,1097813,462.0,328.0,0.151515,1,1,0.0
35715480,12899778,1102546,1054.0,877.0,0.097723,1,1,0.0
35715481,12899778,1068655,488.0,371.0,0.086066,1,1,0.0


# step6 : 学習できる形にするgt:0,gt:1

In [18]:
tar = pd.read_parquet(TRAIN_DIR+'test_labels.parquet')
tar = tar.loc[ tar['type']=='clicks' ]
tar['item'] = tar['ground_truth'].explode().astype('int32')
tar['click'] = 1

tar = tar.rename(columns={'session':'user'})
tar

,user,type,ground_truth,item,click
0,11098528,clicks,[1679529],1679529,1
3,11098529,clicks,[1105029],1105029,1
6,11098532,clicks,[1596491],1596491,1
7,11098533,clicks,[1417450],1417450,1
10,11098534,clicks,[908024],908024,1
...,...,...,...,...,...
2212687,12899774,clicks,[1399483],1399483,1
2212688,12899775,clicks,[1760714],1760714,1
2212689,12899776,clicks,[1737908],1737908,1
2212690,12899777,clicks,[384045],384045,1


In [19]:
tar

,user,type,ground_truth,item,click
0,11098528,clicks,[1679529],1679529,1
3,11098529,clicks,[1105029],1105029,1
6,11098532,clicks,[1596491],1596491,1
7,11098533,clicks,[1417450],1417450,1
10,11098534,clicks,[908024],908024,1
...,...,...,...,...,...
2212687,12899774,clicks,[1399483],1399483,1
2212688,12899775,clicks,[1760714],1760714,1
2212689,12899776,clicks,[1737908],1737908,1
2212690,12899777,clicks,[384045],384045,1


In [20]:
tar = tar[['user', 'item', 'click']]
tar

,user,item,click
0,11098528,1679529,1
3,11098529,1105029,1
6,11098532,1596491,1
7,11098533,1417450,1
10,11098534,908024,1
...,...,...,...
2212687,12899774,1399483,1
2212688,12899775,1760714,1
2212689,12899776,1737908,1
2212690,12899777,384045,1


In [21]:
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)

In [22]:
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio,click
0,11098528,588923,23956.0,14293.0,0.124937,1,1,0.0,0.0
1,11098528,1732105,10121.0,5565.0,0.280802,1,1,0.0,0.0
2,11098528,571762,18185.0,11918.0,0.125763,1,1,0.0,0.0
3,11098528,884502,30419.0,17385.0,0.135738,1,1,0.0,0.0
4,11098528,1157882,27559.0,15556.0,0.137269,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
35715478,12899778,13568,855.0,527.0,0.173099,1,1,0.0,0.0
35715479,12899778,1097813,462.0,328.0,0.151515,1,1,0.0,0.0
35715480,12899778,1102546,1054.0,877.0,0.097723,1,1,0.0,0.0
35715481,12899778,1068655,488.0,371.0,0.086066,1,1,0.0,0.0


# Training

In [23]:
FEATURES = list(candidates.columns[2:8])
FEATURES

['item_item_count',
 'item_user_count',
 'item_by_count',
 'user_user_count',
 'user_item_count',
 'user_buy_ratio']

In [24]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
import pickle

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):
    print(f'=====fold : {fold}=====')

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    #参考　https://knuu.github.io/ltr_by_lightgbm.html

    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'lambdarank_truncation_level': 10,
        'ndcg_eval_at': [20],
        'n_estimators': 10000,
        'boosting_type': 'gbdt',
        'random_state': 0,
    }

    train = lgb.Dataset(X_train, y_train, group=candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values )
    valid = lgb.Dataset(X_valid, y_valid, group=candidates.loc[valid_idx,:].groupby('user')['item'].agg('count').values )
     
     
    model = lgb.train(
        params, train, valid_sets=valid, 
        early_stopping_rounds=50, #50
        verbose_eval=5  # 10 round毎に metric を表示
    )
    
    #model.save_model(OUTPUT_DIR+f'chris_baseline_LGBM_fold{fold}_click.xgb')
    pickle.dump(model, open(OUTPUT_DIR + f'lgb_model_{fold}.pkl','wb'))

=====fold : 0=====


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[5]	valid_0's ndcg@20: 0.849588
[10]	valid_0's ndcg@20: 0.848246
[15]	valid_0's ndcg@20: 0.848343
[20]	valid_0's ndcg@20: 0.848503
[25]	valid_0's ndcg@20: 0.848291
[30]	valid_0's ndcg@20: 0.848205
[35]	valid_0's ndcg@20: 0.848173
[40]	valid_0's ndcg@20: 0.847794
[45]	valid_0's ndcg@20: 0.847863
[50]	valid_0's ndcg@20: 0.847914
Early stopping, best iteration is:
[1]	valid_0's ndcg@20: 0.852812
=====fold : 1=====
Training until validation scores don't improve for 50 rounds.
[5]	valid_0's ndcg@20: 0.849342
[10]	valid_0's ndcg@20: 0.848972
[15]	valid_0's ndcg@20: 0.849235
[20]	valid_0's ndcg@20: 0.848957
[25]	valid_0's ndcg@20: 0.849028
[30]	valid_0's ndcg@20: 0.848942
[35]	valid_0's ndcg@20: 0.848616
[40]	valid_0's ndcg@20: 0.848642
[45]	valid_0's ndcg@20: 0.84876
[50]	valid_0's ndcg@20: 0.848662
Early stopping, best iteration is:
[1]	valid_0's ndcg@20: 0.853409
=====fold : 2=====
Training until validation scores don't improve 

In [55]:
#メモリ節約
#del candidate, train, valid

#Inference

In [27]:
TEST_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-chunk-data-inparquet-format/'

In [28]:
import glob

type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(TEST_DIR + 'test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

df_test = load_test()
print('Test data has shape',df_test.shape)
df_test.head()

Test data has shape (6928123, 4)


,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


# 再step1 sessionのごとにitem候補生成 : test 

In [29]:
%%time

test_candidates = df_test.groupby('session').apply(generate_candidate).explode()
test_candidates

CPU times: user 2min 33s, sys: 2.09 s, total: 2min 35s
Wall time: 2min 34s


session
12899779     941596
12899779     731692
12899779    1340695
12899779    1790770
12899779       3295
             ...   
14571581     984794
14571581    1251890
14571581     990293
14571581    1072049
14571581    1764910
Length: 33159186, dtype: object

In [30]:
test_candidates.name = 'item'
test_candidates = test_candidates.to_frame().reset_index()
test_candidates

,session,item
0,12899779,941596
1,12899779,731692
2,12899779,1340695
3,12899779,1790770
4,12899779,3295
...,...,...
33159181,14571581,984794
33159182,14571581,1251890
33159183,14571581,990293
33159184,14571581,1072049


# 再step2 item特徴 : all_train + test

In [31]:
%%time
all_train = pd.merge(df_train, df_val, on=['session','aid','type'], how='left')
all_item_features = pd.merge(all_train, df_test, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
all_item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
all_item_features.to_parquet('all_item_features.pqt')
all_item_features

CPU times: user 4min 54s, sys: 31.1 s, total: 5min 25s
Wall time: 5min 23s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


In [32]:
#del all_train
#gc.collect()

# 再step3 user特徴 :test

In [33]:
test_user_features = df_test.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
test_user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
test_user_features.to_parquet('test_user_features.pqt')
test_user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
12899779,1,1,0.000000
12899780,5,4,0.000000
12899781,11,5,0.090909
12899782,70,38,0.457143
12899783,11,9,0.000000
...,...,...,...
14571577,1,1,0.000000
14571578,1,1,0.000000
14571579,1,1,0.000000


# 再step5 : 再step1に 再step2,3を追加

In [34]:
test_candidates = test_candidates.rename(columns={'session':'user'})

#item_features = pd.read_parquet('item_features.pqt')
test_candidates = test_candidates.merge(all_item_features, left_on='item', right_index=True, how='left').fillna(-1)
#user_features = pd.read_parquet('user_features.pqt')
test_candidates = test_candidates.merge(test_user_features, left_on='user', right_index=True, how='left').fillna(-1)
test_candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio
0,12899779,941596,68.0,44.0,0.073529,1,1,0.0
1,12899779,731692,85.0,42.0,0.188235,1,1,0.0
2,12899779,1340695,11.0,8.0,0.090909,1,1,0.0
3,12899779,1790770,30.0,20.0,0.133333,1,1,0.0
4,12899779,3295,78.0,66.0,0.012821,1,1,0.0
...,...,...,...,...,...,...,...,...
33159181,14571581,984794,846.0,623.0,0.054374,1,1,0.0
33159182,14571581,1251890,21958.0,14224.0,0.050824,1,1,0.0
33159183,14571581,990293,3958.0,2533.0,0.077312,1,1,0.0
33159184,14571581,1072049,503.0,295.0,0.174950,1,1,0.0


In [35]:
test_candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio
0,12899779,941596,68.0,44.0,0.073529,1,1,0.0
1,12899779,731692,85.0,42.0,0.188235,1,1,0.0
2,12899779,1340695,11.0,8.0,0.090909,1,1,0.0
3,12899779,1790770,30.0,20.0,0.133333,1,1,0.0
4,12899779,3295,78.0,66.0,0.012821,1,1,0.0
...,...,...,...,...,...,...,...,...
33159181,14571581,984794,846.0,623.0,0.054374,1,1,0.0
33159182,14571581,1251890,21958.0,14224.0,0.050824,1,1,0.0
33159183,14571581,990293,3958.0,2533.0,0.077312,1,1,0.0
33159184,14571581,1072049,503.0,295.0,0.174950,1,1,0.0


#pred

In [47]:
import numpy as np

preds = np.zeros(len(test_candidates))
for fold in range(5):
    with open(OUTPUT_DIR + f'lgb_model_{fold}.pkl', 'rb') as f:
      model = pickle.load(f)
    preds += model.predict(test_candidates[FEATURES], num_iteration=model.best_iteration)/5

predictions = test_candidates[['user','item']].copy()
predictions['pred'] = preds
predictions

,user,item,pred
0,12899779,941596,-0.002090
1,12899779,731692,-0.002090
2,12899779,1340695,-0.002090
3,12899779,1790770,-0.002090
4,12899779,3295,-0.001462
...,...,...,...
33159181,14571581,984794,-0.002090
33159182,14571581,1251890,0.002154
33159183,14571581,990293,-0.002090
33159184,14571581,1072049,-0.002090


In [48]:
predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
predictions

,user,item,pred
0,12899779,3295,-0.001462
1,12899779,45290,-0.001462
2,12899779,225209,-0.001462
3,12899779,985764,-0.001462
4,12899779,1019683,-0.001462
...,...,...,...
33159181,14571581,1571699,-0.002090
33159182,14571581,1550662,-0.002090
33159183,14571581,984794,-0.002090
33159184,14571581,990293,-0.002090


In [49]:
predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
predictions

,user,item,pred,n
0,12899779,3295,-0.001462,0
1,12899779,45290,-0.001462,1
2,12899779,225209,-0.001462,2
3,12899779,985764,-0.001462,3
4,12899779,1019683,-0.001462,4
...,...,...,...,...
33159181,14571581,1571699,-0.002090,15
33159182,14571581,1550662,-0.002090,16
33159183,14571581,984794,-0.002090,17
33159184,14571581,990293,-0.002090,18


In [50]:
predictions = predictions.loc[predictions.n<20]
predictions

,user,item,pred,n
0,12899779,3295,-0.001462,0
1,12899779,45290,-0.001462,1
2,12899779,225209,-0.001462,2
3,12899779,985764,-0.001462,3
4,12899779,1019683,-0.001462,4
...,...,...,...,...
33159181,14571581,1571699,-0.002090,15
33159182,14571581,1550662,-0.002090,16
33159183,14571581,984794,-0.002090,17
33159184,14571581,990293,-0.002090,18


In [51]:
sub = predictions.groupby('user').item.apply(list)
sub = sub.to_frame().reset_index()
sub

,user,item
0,12899779,"[3295, 45290, 225209, 985764, 1019683, 1606540..."
1,12899780,"[1660665, 1502122, 889686, 1758603, 487136, 63..."
2,12899781,"[1583317, 950718, 1008624, 1242608, 782161, 18..."
3,12899782,"[1699932, 1675581, 1714681, 356923, 889671, 40..."
4,12899783,"[904889, 1806816, 302318, 1274701, 884774, 163..."
...,...,...
1671798,14571577,"[1768884, 195833, 1289343, 459882, 1368814, 13..."
1671799,14571578,"[1139638, 977826, 1671592, 1536592, 1005419, 1..."
1671800,14571579,"[1750859, 1209992, 1550479, 785544, 51363, 702..."
1671801,14571580,"[477239, 826148, 679257, 1117970, 1682397, 162..."


In [52]:
sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
sub

,user,item
0,12899779,3295 45290 225209 985764 1019683 1606540 11083...
1,12899780,1660665 1502122 889686 1758603 487136 636101 1...
2,12899781,1583317 950718 1008624 1242608 782161 1853288 ...
3,12899782,1699932 1675581 1714681 356923 889671 406001 5...
4,12899783,904889 1806816 302318 1274701 884774 1632267 8...
...,...,...
1671798,14571577,1768884 195833 1289343 459882 1368814 1304792 ...
1671799,14571578,1139638 977826 1671592 1536592 1005419 1248364...
1671800,14571579,1750859 1209992 1550479 785544 51363 702275 70...
1671801,14571580,477239 826148 679257 1117970 1682397 1627186 4...


In [53]:
sub.columns = ['session_type','labels']
sub.session_type = sub.session_type.astype('str')+ '_clicks'
sub

,session_type,labels
0,12899779_clicks,3295 45290 225209 985764 1019683 1606540 11083...
1,12899780_clicks,1660665 1502122 889686 1758603 487136 636101 1...
2,12899781_clicks,1583317 950718 1008624 1242608 782161 1853288 ...
3,12899782_clicks,1699932 1675581 1714681 356923 889671 406001 5...
4,12899783_clicks,904889 1806816 302318 1274701 884774 1632267 8...
...,...,...
1671798,14571577_clicks,1768884 195833 1289343 459882 1368814 1304792 ...
1671799,14571578_clicks,1139638 977826 1671592 1536592 1005419 1248364...
1671800,14571579_clicks,1750859 1209992 1550479 785544 51363 702275 70...
1671801,14571580_clicks,477239 826148 679257 1117970 1682397 1627186 4...


# save

In [54]:
with open(OUTPUT_DIR+'chris_reranker_baseline_click_preds.pkl','wb') as f:
  pickle.dump(sub, f)